<a href="https://colab.research.google.com/github/kurtzace/tax_helper/blob/main/Tax_Computation_Interests_CapitalGains_Dividends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bank Statements

### get interests and dividends

In [ ]:
import pandas as pd
deposits_bobi = pd.read_csv('deposits_bobi.csv',thousands=',')
deposits_icici = pd.read_csv('deposits_icici.csv',thousands=',')
deposits_hdfc = pd.read_csv('deposits_hdfc.csv',thousands=',')
deposits_bobi.columns

Index(['DATE', 'NARRATION', 'CHQ.NO.', 'WITHDRAWAL(DR)', 'DEPOSIT(CR)',
       'BALANCE(INR)'],
      dtype='object')

In [ ]:
print(deposits_icici.columns)
print(deposits_hdfc.columns)

Index(['S No.', 'Value Date', 'Transaction Date', 'Cheque Number',
       'Transaction Remarks', 'Withdrawal Amount (INR )',
       'Deposit Amount (INR )', 'Balance (INR )'],
      dtype='object')
Index(['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.',
       'Deposit Amt.', 'Closing Balance'],
      dtype='object')


In [ ]:
deposits_icici.drop(columns=['S No.','Value Date'], axis=1, inplace=True)
deposits_hdfc.drop(columns=['Value Dt'], axis=1, inplace=True)
print(deposits_icici.columns)
print(deposits_hdfc.columns)

Index(['Transaction Date', 'Cheque Number', 'Transaction Remarks',
       'Withdrawal Amount (INR )', 'Deposit Amount (INR )', 'Balance (INR )'],
      dtype='object')
Index(['Date', 'Narration', 'Chq./Ref.No.', 'Withdrawal Amt.', 'Deposit Amt.',
       'Closing Balance'],
      dtype='object')


In [ ]:
deposits_icici.columns=['date','chequenumber','description','debit','credit', 'balance']
deposits_bobi.columns = ['date','description','chequenumber','debit','credit', 'balance']
deposits_hdfc.columns=['date', 'description','chequenumber','debit','credit', 'balance']
#deposits.replace("", float("NaN"), inplace=True)
#deposits.dropna()

def fixDataSet(dep):
   dep = dep.dropna(subset=['credit']) #only looking at deposits
   dep["date"] = pd.to_datetime(dep["date"], dayfirst = True)
   dep = dep[(dep["date"]>='2020-04-01' )] #within FY range
   dep = dep[( dep["date"]<='2021-03-31' )]

fixDataSet(deposits_bobi)
fixDataSet(deposits_icici)
fixDataSet(deposits_hdfc)
deposits_bobi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          47 non-null     object 
 1   description   47 non-null     object 
 2   chequenumber  0 non-null      float64
 3   debit         0 non-null      float64
 4   credit        47 non-null     float64
 5   balance       47 non-null     object 
dtypes: float64(3), object(3)
memory usage: 2.3+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
deposits_icici.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          94 non-null     object 
 1   chequenumber  94 non-null     object 
 2   description   95 non-null     object 
 3   debit         94 non-null     float64
 4   credit        94 non-null     float64
 5   balance       94 non-null     float64
dtypes: float64(3), object(3)
memory usage: 4.6+ KB


In [ ]:
deposits_hdfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          31 non-null     object 
 1   description   31 non-null     object 
 2   chequenumber  31 non-null     int64  
 3   debit         0 non-null      float64
 4   credit        31 non-null     float64
 5   balance       31 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.6+ KB


In [ ]:
#fix bobi's balance either in 595.43Cr or 1,595.43Dr format
deposits_bobi['balance']=pd.to_numeric(deposits_bobi['balance'].str.replace('[CD]r','').str.replace(',','').str.strip())

In [ ]:
deposits =  pd.concat([deposits_bobi,deposits_hdfc,deposits_icici])

In [ ]:
interests = deposits[deposits["description"].str.lower().str.contains(r'int\.') | deposits["description"].str.contains("CREDIT INTEREST")]

In [ ]:
print("Uncomment below for interests sum")
#interests["credit"].sum()

Uncomment below for interests sum


In [ ]:
dividends = deposits[deposits["description"].str.contains(r'ACH Credit/') | deposits["description"].str.contains("ACH/")]
dividends.to_csv("dividends.csv")
print("Uncomment below for dividends sum")
#dividends["credit"].sum()

Uncomment below for dividends sum


In [ ]:
upiDeposits = deposits[deposits["description"].str.lower().str.contains(r'UPI/') ]

## API

### example get dividents from Yahoo for POKARNA of NSE

In [ ]:
import requests

In [ ]:
## get dividents from Yahoo
range = '1y'
components = ['NS','POKARNA']
url = f'https://query1.finance.yahoo.com/v8/finance/chart/{components[1]}.{components[0]}?formatted=true&crumb=aCUA41Ad5Lt&lang=en-US&region=US&interval=3mo&events=div|split&range={range}&corsDomain=finance.yahoo.com'
r = requests.get(url)
r.json()['chart']['result'][0]['events']

{'dividends': {'1598898600': {'amount': 0.6, 'date': 1600659900},
  '1606761000': {'amount': 0.6, 'date': 1600746300}}}

## get rate on 31st Jan 2018
[Reason](https://learn.quicko.com/long-term-capital-gain-tax-equity-shares)

In [ ]:
url = f'https://query2.finance.yahoo.com/v8/finance/chart/{components[1]}.{components[0]}?formatted=true&crumb=svKKlabb5h2&lang=en-IN&region=IN&includeAdjustedClose=true&interval=1d&period1=1517356800&period2=1517443200&events=div|split&useYfid=true&corsDomain=in.finance.yahoo.com'

r = requests.get(url)
r.json()['chart']['result'][0]['indicators']['quote']

[{'close': [218.75],
  'high': [222.0],
  'low': [215.0],
  'open': [220.64999389648438],
  'volume': [25980]}]